In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/detect_train.npy');
path_to_labels = path.join(root_folder,'data/detect_labels.npy');
path_to_train_data1 = path.join(root_folder,'data/detect_train1.npy');
path_to_labels1 = path.join(root_folder,'data/detect_labels1.npy');

path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 20;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
if (len(torch_labels.size()) == 2):
    torch_labels = torch.squeeze(torch_labels)
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


798792

In [8]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)


%pdb on

Automatic pdb calling has been turned ON


In [9]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
exp_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[2, 7, 12, 17], gamma=0.1)

for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, train_loader)
  cnn.Test(net, device, criterion, test_loader)


Train Epoch: 0 [0/3195168 (0%)]	Loss: 0.74004900455474853516
Train Epoch: 0 [450000/3195168 (14%)]	Loss: 0.05073631182312965393
Train Epoch: 0 [900000/3195168 (28%)]	Loss: 0.03911453858017921448
Train Epoch: 0 [1350000/3195168 (42%)]	Loss: 0.03976726904511451721
Train Epoch: 0 [1800000/3195168 (56%)]	Loss: 0.03657329827547073364
Train Epoch: 0 [2250000/3195168 (70%)]	Loss: 0.03205578029155731201
Train Epoch: 0 [2700000/3195168 (84%)]	Loss: 0.03703232482075691223
Train Epoch: 0 [3150000/3195168 (98%)]	Loss: 0.03264973312616348267

Test set: Average loss: 0.00000213936414183991, Accuracy: 3158709/3195168 (98.85893323919117392506%)


Test set: Average loss: 0.00000215254181057389, Accuracy: 789709/798792 (98.86290799106650695194%)

Train Epoch: 1 [0/3195168 (0%)]	Loss: 0.05002133920788764954
Train Epoch: 1 [450000/3195168 (14%)]	Loss: 0.03348682820796966553
Train Epoch: 1 [900000/3195168 (28%)]	Loss: 0.02821048535406589508
Train Epoch: 1 [1350000/3195168 (42%)]	Loss: 0.0327927358448505401

Train Epoch: 11 [450000/3195168 (14%)]	Loss: 0.03072862699627876282
Train Epoch: 11 [900000/3195168 (28%)]	Loss: 0.02668058872222900391
Train Epoch: 11 [1350000/3195168 (42%)]	Loss: 0.02854548394680023193
Train Epoch: 11 [1800000/3195168 (56%)]	Loss: 0.03009344451129436493
Train Epoch: 11 [2250000/3195168 (70%)]	Loss: 0.02553318068385124207
Train Epoch: 11 [2700000/3195168 (84%)]	Loss: 0.02864065393805503845
Train Epoch: 11 [3150000/3195168 (98%)]	Loss: 0.03010469675064086914

Test set: Average loss: 0.00000176142339114449, Accuracy: 3164653/3195168 (99.04496414586024854998%)


Test set: Average loss: 0.00000179591029336734, Accuracy: 791177/798792 (99.04668549509759145622%)

Train Epoch: 12 [0/3195168 (0%)]	Loss: 0.02713610418140888214
Train Epoch: 12 [450000/3195168 (14%)]	Loss: 0.02634244225919246674
Train Epoch: 12 [900000/3195168 (28%)]	Loss: 0.02785749733448028564
Train Epoch: 12 [1350000/3195168 (42%)]	Loss: 0.02609078399837017059
Train Epoch: 12 [1800000/3195168 (56%)]	Loss: 0.

In [10]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00000179494077201525, Accuracy: 791180/798792 (99.04706106220392314299%)

